In [1]:
import json
import pandas as pd
import os

In [3]:
# category = '["American Football","Aquatics","Archery","Athletics","Australian Rules Football","Auto Racing","Badminton","Baseball and Softball","Basketball","Biathlon","Billiards","Bobsleigh and Skeleton","Bowling","Boxing","Canadian Football","Canoeing","Chess and Cards","Cricket","Curling","Cycling","DanceSport","Darts","Dragon Boat","Equestrianism","eSports","Fencing","Field Hockey","Football","Gaelic Football","Frisbee","Golf","Gymnastics","Handball","Ice Hockey","Judo","Kabaddi","Karate","Kickboxing","Lacrosse","Luge","MMA","Modern Pentathlon","Motorcycle Sports","Muaythai","Other Sport","Powerboating","Racquetball","Roller Sports","Rowing","Rugby League","Rugby Union","Sailing","Sanda","Shooting","Skating","Ski Mountaineering","Skiing","Sport Climbing","Squash","Surfing","Table Tennis","Taekwondo","Tennis","Triathlon","Volleyball","Weightlifting","Wrestling","Wushu"]'

# prompt = 'You are an expert in text classification with a specialization in the sports domain. Please follow these specific instructions: 1. Include the sport types for the competition. The sport types must belong in the following categories: ["American Football", "Aquatics", "Archery", "Athletics", "Australian Rules Football", "Auto Racing", "Badminton", "Baseball and Softball", "Basketball", "Biathlon", "Billiards", "Bobsleigh and Skeleton", "Bowling", "Boxing", "Canadian Football", "Canoeing", "Chess and Cards", "Cricket", "Curling", "Cycling", "DanceSport", "Darts", "Dragon Boat", "Equestrianism", "eSports", "Fencing", "Field Hockey", "Football", "Gaelic Football", "Frisbee", "Golf", "Gymnastics", "Handball", "Ice Hockey", "Judo", "Kabaddi", "Karate", "Kickboxing", "Lacrosse", "Luge", "MMA", "Modern Pentathlon", "Motorcycle Sports", "Muaythai", "Other Sport", "Powerboating", "Racquetball", "Roller Sports", "Rowing", "Rugby League", "Rugby Union", "Sailing", "Sanda", "Shooting", "Skating", "Ski Mountaineering", "Skiing", "Sport Climbing", "Squash", "Surfing", "Table Tennis", "Taekwondo", "Tennis", "Triathlon", "Volleyball", "Weightlifting", "Wrestling", "Wushu"]. Give all possible sport types if the text is not clear. However, if the text is clear and can be classified into a certain sport type, only give one type. For example, "NORWAY VS DENMARK NORWAY VS DENMARK" can be "Football, Basketball, Volleyball". 2. Each result should be a string object indicating the sport types. For example: "Football" or "Football, Basketball". 3. Only give final string object. I will save them in python.'
category = '["American Football", "Aquatics", "Archery", "Athletics", "Australian Rules Football", "Auto Racing", "Badminton", "Baseball and Softball", "Basketball", "Biathlon", "Billiards", "Bobsleigh and Skeleton", "Bowling", "Boxing", "Canadian Football", "Canoeing", "Chess and Cards", "Cricket", "Curling", "Cycling", "DanceSport", "Darts", "Dragon Boat", "Equestrianism", "eSports", "Fencing", "Field Hockey", "Football", "Gaelic Football", "Frisbee", "Golf", "Gymnastics", "Handball", "Ice Hockey", "Judo", "Kabaddi", "Karate", "Kickboxing", "Lacrosse", "Luge", "MMA", "Modern Pentathlon", "Motorcycle Sports", "Muaythai", "Other Sport", "Powerboating", "Racquetball", "Roller Sports", "Rowing", "Rugby League", "Rugby Union", "Sailing", "Sanda", "Shooting", "Skating", "Ski Mountaineering", "Skiing", "Sport Climbing", "Squash", "Surfing", "Table Tennis", "Taekwondo", "Tennis", "Triathlon", "Volleyball", "Weightlifting", "Wrestling", "Wushu"]'
prompt = '''You are an expert in text classification with a specialization in the sports domain. Please follow these specific instructions:
1. Include the sport types for the competition: The sport types must belong to the given categories. 
   - For unclear texts: List all possible sport types that match the text (e.g., "Football, Basketball, Volleyball").
   - For clear cases: When the text clearly identifies a single sport, return only that specific sport type (e.g., "Football").
2. Output format:
   - Return only the final string object, containing sport types as a comma-separated list. 
   - For clear text like "UEFA Champions League" → "Football"
   - For unclear text like "NORWAY VS DENMARK" → "Football, Basketball, Volleyball"
3. Only provide the final string output, as I will save it in Python.'''

In [9]:
def dataset_jsonl_transfer(origin_path, new_path):
    """
    将原始数据集转换为大模型微调所需数据格式的新数据集
    """
    messages = []

    # 读取旧的JSONL文件
    df = pd.read_excel(origin_path)
    for _, row in df.iterrows():
        context = row["Text"]
        # genre_category = row["Genre_Cate"]
        # category = row["Category"]
        # genre_label = row["Genre"]
        label = row["Standardized_Sport"] # row["Speciality"]
        # print(label)
        message = {
            "instruction": prompt,
            "input": f"Context: {context}, Categories: {category}",
            "output": label,
        }
        messages.append(message)

    # 保存重构后的JSONL文件
    with open(new_path, "w", encoding="utf-8") as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + "\n")

In [11]:
# 加载、处理数据集和测试集
train_dataset_path = "./data/classification/train_data_4k.xlsx"
train_jsonl_new_path = "./data/classification/train_4k.jsonl"
# test_dataset_path = "test_data_6k.xlsx"
# test_jsonl_new_path = "new_test_6k.jsonl"

# train_dataset_path = "train.jsonl"
# test_dataset_path = "test.jsonl"
# train_jsonl_new_path = "new_train.jsonl"
# test_jsonl_new_path = "new_test.jsonl"

if not os.path.exists(train_jsonl_new_path):
    dataset_jsonl_transfer(train_dataset_path, train_jsonl_new_path)
# if not os.path.exists(test_jsonl_new_path):
    # dataset_jsonl_transfer(test_dataset_path, test_jsonl_new_path)

In [13]:
train_df = pd.read_json(train_jsonl_new_path, lines=True)
# test_df = pd.read_json(test_jsonl_new_path, lines=True)

# train_df = pd.read_json(train_jsonl_new_path, lines=True)
# test_df = pd.read_json(test_jsonl_new_path, lines=True)

In [15]:
train_df.shape

(4044, 3)

In [17]:
from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
import torch

# 在modelscope上下载Qwen模型到本地目录下
# model_dir = snapshot_download("qwen/Qwen2.5-1.5B-Instruct", cache_dir="./", revision="master")

2024-11-11 17:32:21,315 - modelscope - INFO - PyTorch version 2.4.1+cu118 Found.
2024-11-11 17:32:21,315 - modelscope - INFO - Loading ast index from C:\Users\wangxiaoyang\.cache\modelscope\ast_indexer
2024-11-11 17:32:21,377 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 606b6fdc4b26d9e313c2ded5bec68f0b and a total number of 976 components indexed
D:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained("./qwen/Qwen2___5-1___5B-Instruct/", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./qwen/Qwen2___5-1___5B-Instruct/", device_map="auto", torch_dtype=torch.bfloat16)
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
temp_train = train_df.groupby('output').apply(
    lambda x: x.sample(n=25, random_state=42) if len(x) > 25 else x
).reset_index(drop=True)

In [58]:
temp_train

,instruction,input,output
0,You are an expert in text classification with ...,Context: 2019 NFL (REPLAY) 'DALLAS COWBOYS VS ...,American Football
1,You are an expert in text classification with ...,Context: 2019 NCAA COLLEGE FOOTBALL PLAYOFF CH...,American Football
2,You are an expert in text classification with ...,Context: 2019 NFL 'GREEN BAY PACKERS VS MINNES...,American Football
3,You are an expert in text classification with ...,Context: 2019 ROSE BOWL (LIVE) 'OHIO STATE BUC...,American Football
4,You are an expert in text classification with ...,Context: NFL SEASON 2018-19 'BUFFALO BILLS VS ...,American Football
...,...,...,...
957,You are an expert in text classification with ...,Context: MPL MY S8WEEK 4 DAY 1 MOBILE LEGENDS ...,eSports
958,You are an expert in text classification with ...,Context: MOBILE LEGENDSSESI KE-4 REBORN VS RSG...,eSports
959,You are an expert in text classification with ...,Context: FIFA ENATIONS STAY AND PLAY CUP 25 AP...,eSports
960,You are an expert in text classification with ...,Context: THE NATIONALSTEKKEN LEAGUE DAY 9 THE ...,eSports


In [21]:
def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{prompt}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = (
        instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = (
        [-100] * len(instruction["input_ids"])
        + response["input_ids"]
        + [tokenizer.pad_token_id]
    )
    # print("Labels:", labels)
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [23]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)

Map: 100%|█████████████████████████████████████████████████████████████████| 4044/4044 [00:07<00:00, 564.49 examples/s]


In [25]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

model = get_peft_model(model, config)

In [27]:
args = TrainingArguments(
    output_dir="./output/qwen2.5-1.5B-finetuned-classification",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=1,
    num_train_epochs=1,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

In [35]:
from transformers import Trainer, TrainingArguments, TrainerCallback

class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        # 获取 labels 并打印
        labels = inputs.get("labels")
        print("Labels:", labels)

        # 调用父类的 training_step
        return super().training_step(model, inputs)

# 定义训练参数
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=1,
)

# 初始化 Trainer 并传入自定义 Callback
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    # callbacks=[PrintLabelsCallback]
)

# 开始训练
trainer.train()


Labels: tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]], device='cuda:0')


D:\anaconda3\envs\pytorch\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,0.000000


Labels: tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]], device='cuda:0')
Labels: tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]], device='cuda:0')


KeyboardInterrupt: 

In [35]:
from swanlab.integration.huggingface import SwanLabCallback
import swanlab

swanlab_callback = SwanLabCallback(
    project="Qwen2___5-finetune",
    experiment_name="Qwen2.5-1.5B-Instruct",
    description="使用通义千问Qwen2.5-1.5B-Instruct模型在Foreign体育数据集上微调-Classification",
    config={
        "model": "qwen/Qwen2.5-1.5B-Instruct",
        "dataset": "data-foreign-4k",
    },
)

In [29]:
model = model.to("cuda")

In [31]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    # callbacks=[swanlab_callback],
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
D:\anaconda3\envs\pytorch\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\anaconda3\envs\pytorch\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:679: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
D:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\checkpoint

Step,Training Loss
1,0.000000


KeyboardInterrupt: 

In [ ]:
1+1

In [40]:
test_df = train_df.head(10)

In [42]:
from tqdm import tqdm

In [44]:
# ====== 训练结束后的预测 ===== #

def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

    return response

model = model.to("cuda")
import time

# 记录开始时间
start_time = time.time()

test_text_list = []
result_list = []
for index, row in tqdm(test_df.iterrows(), desc=f"Predicting", unit="row"):
# for index, row in test_df.iterrows():
    instruction = row["instruction"]
    input_value = row["input"]

    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"},
    ]

    response = predict(messages, model, tokenizer)
    # messages.append({"role": "assistant", "content": f"{response}"})
    # result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
    result_list.append(response)
    test_text_list.append(input_value)
    # test_text_list.append(swanlab.Text(result_text, caption=response))

# 记录结束时间
end_time = time.time()

# 计算总用时
execution_time = end_time - start_time

# swanlab.log({"Prediction": test_text_list})
# swanlab.finish()

Predicting: 0row [00:00, ?row/s]D:\anaconda3\envs\pytorch\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Predicting: 10row [00:08,  1.13row/s]


In [34]:
execution_time

10.680860042572021

In [44]:
test_df.shape

(1184, 3)

In [46]:
train_df.shape

(4736, 3)

In [36]:
test_df['result'] = result_list

In [37]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_df.output, test_df.result)
print(f"Accuracy: {accuracy:.2f}")

TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [ ]:
accuracy

In [40]:
test_df.to_excel("./temp_res.xlsx", index=False)

In [47]:
test_df.output.value_counts()

output
Football                             403
Badminton                             20
Other Competitions(From 20060101)     20
Basketball                            20
Boxing                                20
Volleyball                            20
Tennis                                20
Racing                                20
Ping-Pong                             20
Billiards                             20
Ice/Water Sports(From 20060101)       20
Golf(From 20060101)                   13
Chess And Cards                        8
Bowling                                3
Name: count, dtype: int64

In [60]:
model.save_pretrained("./Qwen2-1___5B-Instruct-Finetuned")
tokenizer.save_pretrained("./Qwen2-1___5B-Instruct-Finetuned")

D:\anaconda3\envs\pytorch\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in ../qwen/Qwen2-1___5B-Instruct/ - will assume that the vocabulary was not modified.
  warnings.warn(


('./Qwen2-1___5B-Instruct-Finetuned\\tokenizer_config.json',
 './Qwen2-1___5B-Instruct-Finetuned\\special_tokens_map.json',
 './Qwen2-1___5B-Instruct-Finetuned\\vocab.json',
 './Qwen2-1___5B-Instruct-Finetuned\\merges.txt',
 './Qwen2-1___5B-Instruct-Finetuned\\added_tokens.json')